In [ ]:
from langgraph.prebuilt import create_react_agent

In [ ]:
import functools
import inspect
from typing import (
    Any,
    Callable,
    Literal,
    Optional,
    Sequence,
    Type,
    TypeVar,
    Union,
    cast,
    get_type_hints,
)

from langchain_core.language_models import (
    BaseChatModel,
    LanguageModelInput,
    LanguageModelLike,
)
from langchain_core.messages import AIMessage, BaseMessage, SystemMessage, ToolMessage
from langchain_core.runnables import (
    Runnable,
    RunnableBinding,
    RunnableConfig,
    RunnableSequence,
)
from langchain_core.tools import BaseTool
from pydantic import BaseModel
from typing_extensions import Annotated, TypedDict

from langgraph.errors import ErrorCode, create_error_message
from langgraph.graph import END, StateGraph
from langgraph.graph.graph import CompiledGraph
from langgraph.graph.message import add_messages
from langgraph.managed import IsLastStep, RemainingSteps
from langgraph.prebuilt.tool_node import ToolNode
from langgraph.store.base import BaseStore
from langgraph.types import Checkpointer, Send
from langgraph.utils.runnable import RunnableCallable

StructuredResponse = Union[dict, BaseModel]
StructuredResponseSchema = Union[dict, type[BaseModel]]
F = TypeVar("F", bound=Callable[..., Any])


# We create the AgentState that we will pass around
# This simply involves a list of messages
# We want steps to return messages to append to the list
# So we annotate the messages attribute with `add_messages` reducer
class AgentState(TypedDict):
    """The state of the agent."""

    messages: Annotated[Sequence[BaseMessage], add_messages]

    is_last_step: IsLastStep

    remaining_steps: RemainingSteps


class AgentStatePydantic(BaseModel):
    """The state of the agent."""

    messages: Annotated[Sequence[BaseMessage], add_messages]

    remaining_steps: RemainingSteps = 25


class AgentStateWithStructuredResponse(AgentState):
    """The state of the agent with a structured response."""

    structured_response: StructuredResponse


class AgentStateWithStructuredResponsePydantic(AgentStatePydantic):
    """The state of the agent with a structured response."""

    structured_response: StructuredResponse


StateSchema = TypeVar("StateSchema", bound=Union[AgentState, AgentStatePydantic])
StateSchemaType = Type[StateSchema]

PROMPT_RUNNABLE_NAME = "Prompt"

Prompt = Union[
    SystemMessage,
    str,
    Callable[[StateSchema], LanguageModelInput],
    Runnable[StateSchema, LanguageModelInput],
]


def _get_state_value(state: StateSchema, key: str, default: Any = None) -> Any:
    return (
        state.get(key, default)
        if isinstance(state, dict)
        else getattr(state, key, default)
    )


def _get_prompt_runnable(prompt: Optional[Prompt]) -> Runnable:
    prompt_runnable: Runnable
    if prompt is None:
        prompt_runnable = RunnableCallable(
            lambda state: _get_state_value(state, "messages"), name=PROMPT_RUNNABLE_NAME
        )
    elif isinstance(prompt, str):
        _system_message: BaseMessage = SystemMessage(content=prompt)
        prompt_runnable = RunnableCallable(
            lambda state: [_system_message] + _get_state_value(state, "messages"),
            name=PROMPT_RUNNABLE_NAME,
        )
    elif isinstance(prompt, SystemMessage):
        prompt_runnable = RunnableCallable(
            lambda state: [prompt] + _get_state_value(state, "messages"),
            name=PROMPT_RUNNABLE_NAME,
        )
    elif inspect.iscoroutinefunction(prompt):
        prompt_runnable = RunnableCallable(
            None,
            prompt,
            name=PROMPT_RUNNABLE_NAME,
        )
    elif callable(prompt):
        prompt_runnable = RunnableCallable(
            prompt,
            name=PROMPT_RUNNABLE_NAME,
        )
    elif isinstance(prompt, Runnable):
        prompt_runnable = prompt
    else:
        raise ValueError(f"Got unexpected type for `prompt`: {type(prompt)}")

    return prompt_runnable


def _convert_modifier_to_prompt(func: F) -> F:
    """Decorator that converts state_modifier kwarg to prompt kwarg."""

    @functools.wraps(func)
    def wrapper(*args: Any, **kwargs: Any) -> Any:
        prompt = kwargs.get("prompt")
        state_modifier = kwargs.pop("state_modifier", None)
        if sum(p is not None for p in (prompt, state_modifier)) > 1:
            raise ValueError(
                "Expected only one of (prompt, state_modifier), got multiple values"
            )

        if state_modifier is not None:
            prompt = state_modifier

        kwargs["prompt"] = prompt
        return func(*args, **kwargs)

    return cast(F, wrapper)


def _should_bind_tools(model: LanguageModelLike, tools: Sequence[BaseTool]) -> bool:
    if isinstance(model, RunnableSequence):
        model = next(
            (
                step
                for step in model.steps
                if isinstance(step, (RunnableBinding, BaseChatModel))
            ),
            model,
        )

    if not isinstance(model, RunnableBinding):
        return True

    if "tools" not in model.kwargs:
        return True

    bound_tools = model.kwargs["tools"]
    if len(tools) != len(bound_tools):
        raise ValueError(
            "Number of tools in the model.bind_tools() and tools passed to create_react_agent must match"
        )

    tool_names = set(tool.name for tool in tools)
    bound_tool_names = set()
    for bound_tool in bound_tools:
        # OpenAI-style tool
        if bound_tool.get("type") == "function":
            bound_tool_name = bound_tool["function"]["name"]
        # Anthropic-style tool
        elif bound_tool.get("name"):
            bound_tool_name = bound_tool["name"]
        else:
            # unknown tool type so we'll ignore it
            continue

        bound_tool_names.add(bound_tool_name)

    if missing_tools := tool_names - bound_tool_names:
        raise ValueError(f"Missing tools '{missing_tools}' in the model.bind_tools()")

    return False


def _get_model(model: LanguageModelLike) -> BaseChatModel:
    """Get the underlying model from a RunnableBinding or return the model itself."""
    if isinstance(model, RunnableSequence):
        model = next(
            (
                step
                for step in model.steps
                if isinstance(step, (RunnableBinding, BaseChatModel))
            ),
            model,
        )

    if isinstance(model, RunnableBinding):
        model = model.bound

    if not isinstance(model, BaseChatModel):
        raise TypeError(
            f"Expected `model` to be a ChatModel or RunnableBinding (e.g. model.bind_tools(...)), got {type(model)}"
        )

    return model


def _validate_chat_history(
    messages: Sequence[BaseMessage],
) -> None:
    """Validate that all tool calls in AIMessages have a corresponding ToolMessage."""
    all_tool_calls = [
        tool_call
        for message in messages
        if isinstance(message, AIMessage)
        for tool_call in message.tool_calls
    ]
    tool_call_ids_with_results = {
        message.tool_call_id for message in messages if isinstance(message, ToolMessage)
    }
    tool_calls_without_results = [
        tool_call
        for tool_call in all_tool_calls
        if tool_call["id"] not in tool_call_ids_with_results
    ]
    if not tool_calls_without_results:
        return

    error_message = create_error_message(
        message="Found AIMessages with tool_calls that do not have a corresponding ToolMessage. "
        f"Here are the first few of those tool calls: {tool_calls_without_results[:3]}.\n\n"
        "Every tool call (LLM requesting to call a tool) in the message history MUST have a corresponding ToolMessage "
        "(result of a tool invocation to return to the LLM) - this is required by most LLM providers.",
        error_code=ErrorCode.INVALID_CHAT_HISTORY,
    )
    raise ValueError(error_message)

### **Membuat Agent Organizer**

In [ ]:
from abc import ABC, abstractmethod
from typing import Dict, Any
from langchain_core.messages import ToolMessage, AIMessage  # pastikan impor ini sesuai struktur proyekmu


class BaseFallbackToolCalling(ABC):
    """Abstract base class for handling fallback tool logic."""

    tool_map: Dict[str, str] = {}

    @classmethod
    @abstractmethod
    def checker(
        cls,
        tool_message: ToolMessage
    ) -> bool:
        """
        Check whether a fallback tool should be triggered based on the tool message.
        """
        pass

    @classmethod
    @abstractmethod
    def tool_call(
        cls,
        prev_tool_call: ToolMessage,
        name: Optional[str] = None
    ) -> Dict[str, Any]:
        """
        Create a new tool call message as a fallback attempt.
        """
        pass

In [ ]:
import uuid
import json


class FallbackToolCalling(BaseFallbackToolCalling):
    tool_map = {
        "text2cypher_retriever": "vector_cypher_retriever"
    }

    @classmethod
    def checker(
        cls,
        tool_message: ToolMessage
    ) -> bool:
        """
        Check whether a fallback tool should be triggered based on the tool message.
        """
        for availabel_tool_name in cls.tool_map.keys():
            if tool_message.name in availabel_tool_name:
                should_call_alternate_tool = not tool_message.artifact.get("is_context_fetched", False)
                return should_call_alternate_tool
        return False

    @classmethod
    def tool_call(
        cls,
        prev_tool_call: ToolMessage,
        name: Optional[str] = None
    ) -> AIMessage:
        """
        Create a new tool call message as a fallback attempt.
        """
        for availabel_tool_name in cls.tool_map.keys():
            if prev_tool_call.tool_calls[0]['name'] in availabel_tool_name:
                alternate_tool = cls.tool_map[availabel_tool_name]

        return AIMessage(
            content=(
                "Tidak dapat menemukan data yang sesuai dengan permintaan query: "
                f"{prev_tool_call.additional_kwargs['function_call']['arguments']} dengan "
                f"menggunakan tool {prev_tool_call.tool_calls[0]['name']}. Mencoba ulang "
                f"pencarian data dengan menggunakan tool alternatif: {alternate_tool}"
            ),
            additional_kwargs={
                "function_call": {
                    "name": alternate_tool,
                    "arguments": prev_tool_call.additional_kwargs["function_call"]["arguments"]
                }
            },
            response_metadata={},
            type=prev_tool_call.type,
            name=name,
            id=f"run-{uuid.uuid4()}-0",
            example=prev_tool_call.example,
            tool_calls=[{
                "name": alternate_tool,
                # Di langchain ada schema parser, coba itu buat parse str ini menjadi dict, dan bukan pakai json.loads()
                "args": json.loads(prev_tool_call.additional_kwargs["function_call"]["arguments"]),
                "id": str(uuid.uuid4()),
                "type": "tool_call"
            }],
            invalid_tool_calls=prev_tool_call.invalid_tool_calls,
            usage_metadata = {
                'input_tokens': 0,
                'output_tokens': 0,
                'total_tokens': 0,
                # Bagian di bawah ini ternyata di hasil ChatOllama tidak ada
                'input_token_details': {
                    'cache_read': 0
                }
            }
        )

In [ ]:
def create_agent(
    model: BaseChatModel,
    tools: Sequence[Union[BaseTool, Callable]],
    *,
    prompt: Optional[Prompt] = None,
    name: Optional[str] = None,
    fallback_tool_calling: Optional[Type[BaseFallbackToolCalling]] = None,
):
    if isinstance(tools, ToolNode):
        tool_classes = list(tools.tools_by_name.values())
    else:
        tool_node = ToolNode(tools)
        # get the tool functions wrapped in a tool class from the ToolNode
        tool_classes = list(tool_node.tools_by_name.values())

    tool_calling_enabled = len(tool_classes) > 0

    if _should_bind_tools(model, tool_classes) and tool_calling_enabled:
        model = cast(BaseChatModel, model).bind_tools(tool_classes)

    model_runnable = _get_prompt_runnable(prompt) | model
    
    def _are_more_steps_needed(state: StateSchema, response: BaseMessage) -> bool:
        has_tool_calls = isinstance(response, AIMessage) and response.tool_calls
        remaining_steps = _get_state_value(state, "remaining_steps", None)
        is_last_step = _get_state_value(state, "is_last_step", False)
        return (
            (remaining_steps is None and is_last_step and has_tool_calls)
            or (remaining_steps is not None and remaining_steps < 2 and has_tool_calls)
        )

    # Define the function that calls the model
    def call_model(state: StateSchema, config: RunnableConfig) -> StateSchema:
        messages = _get_state_value(state, "messages")
        _validate_chat_history(messages)

        ######################## BUATANKU ##########################
        # Cek jika tool sebelumnya gagal mendapatkan data
        last_message = messages[-1]
        if fallback_tool_calling is not None and isinstance(last_message, ToolMessage):
            if fallback_tool_calling.checker(last_message):
                print("FOLLBACK")
                tool_call_message = fallback_tool_calling.tool_call(
                    prev_tool_call=messages[-2], name=name
                )
                return {
                    "messages": [tool_call_message],
                    "steps": [name]
                }
        print("NO FOLLBACK")
        ############################################################
        # TODO: BARU SAMPAI SINI, BELUM TESTING
        
        response = cast(AIMessage, model_runnable.invoke(state, config))
        # add agent name to the AIMessage
        response.name = name

        if _are_more_steps_needed(state, response):
            return {
                "messages": [
                    AIMessage(
                        id=response.id,
                        content=(
                            "Maaf, perlu langkah lebih lanjut untuk memproses "
                            "permintaan ini."
                        ),
                    )
                ],
                "steps": [name]
            }
        # We return a list, because this will get added to the existing list
        return {
            "messages": [response],
            "steps": [name]
        }

    return RunnableCallable(call_model)

### **Membuat Agent Perangkum**

In [ ]:
# Bisa aku modifikasi sehingga menjadi Node Perangkum
# Ini bisa di skip dulu, jika agent di atas sudah jadi,
# maka lanjut ke workflow dulu saja, hingga testing.
# Kerjakan ini sesudah yang lainnya bisa

def generate_structured_response(
    state: StateSchema, config: RunnableConfig
) -> StateSchema:
    # NOTE: we exclude the last message because there is enough information
    # for the LLM to generate the structured response
    messages = _get_state_value(state, "messages")[:-1]
    structured_response_schema = response_format
    if isinstance(response_format, tuple):
        system_prompt, structured_response_schema = response_format
        messages = [SystemMessage(content=system_prompt)] + list(messages)

    model_with_structured_output = _get_model(model).with_structured_output(
        cast(StructuredResponseSchema, structured_response_schema)
    )
    response = model_with_structured_output.invoke(messages, config)
    return {"structured_response": response}

async def agenerate_structured_response(
    state: StateSchema, config: RunnableConfig
) -> StateSchema:
    # NOTE: we exclude the last message because there is enough information
    # for the LLM to generate the structured response
    messages = _get_state_value(state, "messages")[:-1]
    structured_response_schema = response_format
    if isinstance(response_format, tuple):
        system_prompt, structured_response_schema = response_format
        messages = [SystemMessage(content=system_prompt)] + list(messages)

    model_with_structured_output = _get_model(model).with_structured_output(
        cast(StructuredResponseSchema, structured_response_schema)
    )
    response = await model_with_structured_output.ainvoke(messages, config)
    return {"structured_response": response}

### **Membuat Worflow**

In [ ]:
def create_workflow(
    model: Union[str, LanguageModelLike],
    tools: Sequence[Union[BaseTool, Callable]],
    *,
    prompt: Optional[Prompt] = None,
    response_format: Optional[
        Union[StructuredResponseSchema, tuple[str, StructuredResponseSchema]]
    ] = None,  # Sementara, untuk testing
    state_schema: Optional[StateSchemaType] = None,
    config_schema: Optional[Type[Any]] = None,
    checkpointer: Optional[Checkpointer] = None,
    store: Optional[BaseStore] = None,
    name: Optional[str] = None,
    fallback_tool_calling: Optional[Type[BaseFallbackToolCalling]] = None
):
    if state_schema is not None:
        required_keys = {"messages", "remaining_steps"}

        schema_keys = set(get_type_hints(state_schema))
        if missing_keys := required_keys - set(schema_keys):
            raise ValueError(f"Missing required key(s) {missing_keys} in state_schema")

    if state_schema is None:
        state_schema = AgentState  # Pakai AgentStateWithStructuredResponse jika Agent Perangkum sudah jadi

    tool_node = ToolNode(tools)
    
    # JIKA not last_message.tool_calls MAKA ARAHKAN KE NODE PERANGKUM
    # Define the function that determines whether to continue or not
    def should_continue(state: StateSchema) -> Union[str, list]:
        messages = _get_state_value(state, "messages")
        last_message = messages[-1]
        # If there is no function call, then we finish
        if not isinstance(last_message, AIMessage) or not last_message.tool_calls:
            return END if response_format is None else "generate_structured_response"
        # Otherwise if there is, we continue
        else:
            # return "tools"
            # NGGA TAHU KENAPA KALAU PAKAI V2 ITU MALAH ERROR
            tool_calls = [
                tool_node.inject_tool_args(call, state, store)  # type: ignore[arg-type]
                for call in last_message.tool_calls
            ]
            return [Send("tools", [tool_call]) for tool_call in tool_calls]
    
    # MEMBUAT WORKFLOW
    #########################################################################################
    
    # Define a new graph
    workflow = StateGraph(state_schema or AgentState, config_schema=config_schema)

    # Define the two nodes we will cycle between
    workflow.add_node("agent", create_agent(
            model=model,
            tools=tool_node,
            prompt=prompt,
            name="agent",
            fallback_tool_calling=fallback_tool_calling
        )
    )
    workflow.add_node("tools", tool_node)

    # Set the entrypoint as `agent`
    # This means that this node is the first one called
    workflow.set_entry_point("agent")
    
    # We now add a conditional edge
    workflow.add_conditional_edges(
        # First, we define the start node. We use `agent`.
        # This means these are the edges taken after the `agent` node is called.
        "agent",
        # Next, we pass in the function that will determine which node is called next.
        should_continue,
        path_map=["tools", END]
    )

    workflow.add_edge("tools", "agent")

    # Finally, we compile it!
    # This compiles it into a LangChain Runnable,
    # meaning you can use it as you would any other runnable
    return workflow.compile(
        checkpointer=checkpointer,
        store=store,
        name=name
    )

In [ ]:
import os

from pprint import pprint
from dotenv import load_dotenv
from neo4j import GraphDatabase
from langchain_neo4j import Neo4jGraph
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_google_genai import ChatGoogleGenerativeAI
from src.grag.retrievers.vector_cypher.vector_cypher import create_vector_cypher_retriever_tool
from src.grag.retrievers.text2cypher.text2cypher import create_text2cypher_retriever_tool

load_dotenv(".env")

In [ ]:
URI = os.environ["DATABASE_HOST"]
DATABASE = os.environ["DATABASE_SMALL"]
USERNAME = os.environ["DATABASE_USERNAME"]
PASSWORD = os.environ["DATABASE_PASSWORD"]
DATABASE = os.environ["DATABASE_SMALL"]

neo4j_config = {
    "DATABASE_NAME": DATABASE,
    "ARTICLE_VECTOR_INDEX_NAME": os.environ["ARTICLE_VECTOR_INDEX_NAME"],
    "ARTICLE_FULLTEXT_INDEX_NAME": os.environ["ARTICLE_FULLTEXT_INDEX_NAME"],
    "DEFINITION_VECTOR_INDEX_NAME": os.environ["DEFINITION_VECTOR_INDEX_NAME"],
    "DEFINITION_FULLTEXT_INDEX_NAME": os.environ["DEFINITION_FULLTEXT_INDEX_NAME"],
}

# Ngga jadi pakai ini karena Text2Cypher mintannya harus pakai Neo4jGraph
# neo4j_driver = GraphDatabase.driver(uri=URI, auth=(USERNAME, PASSWORD))

neo4j_graph = Neo4jGraph(
    url=URI,
    username=USERNAME,
    password=PASSWORD,
    database=DATABASE,
    enhanced_schema=True
)

neo4j_driver = neo4j_graph._driver  # Ambil driver nya kaya gini

embedder_model = HuggingFaceEmbeddings(model_name=os.environ["EMBEDDING_MODEL"])

llm = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash",
    temperature=0.0,
    api_key=os.environ["GOOGLE_API_KEY"]
)

In [ ]:
vector_cypher_retriever = create_vector_cypher_retriever_tool(
    embedder_model=embedder_model,
    neo4j_driver=neo4j_driver,
    neo4j_config=neo4j_config,
    top_k_initial_article=5,
    total_article_limit=10,
    total_definition_limit=5
)

text2cypher_retriever = create_text2cypher_retriever_tool(
    neo4j_graph=neo4j_graph,
    embedder_model=embedder_model,
    cypher_llm=llm,
    qa_llm=llm,
    skip_qa_llm=True,
    verbose=False
)

In [ ]:
# Harus memastikan hanya memanggil 1 tool dalam sebuah pemanggilan

SYSTEM_PROMPT = """You are an intelligent assistant that can query a legal graph database (Neo4j) using `text2cypher_retriever` or `vector_cypher_retriever`. Your goal is to accurately `Answer` user queries by utilizing some tools to fetch relevant information.

### Instructions:
1. **Understand the User Query**
   - Carefully analyze the user's question and determine the best approach to retrieve the required information.

2. **Use Available Tools**
   - If the user asks **general questions** that can't be writed in Neo4j Cypher, use `vector_cypher_retriever`.
   - If the user asks for **regulation structure, relationships, or anything that can be represented as Neo4j Cypher**, use `text2cypher_retriever`.
   - Make sure to only call 1 tool in a call

3. **Maintain Accuracy and Completeness**
   - Your default language is English, but you should `Answer` the user query in the same language as the query.
   - Always provide precise and concise `Answer` based on the retrieved data.
   - If the retrieved data contains legal articles with subsections, structure them in a markdown list format.
   - Ensure that your final `Answer` is well-formatted in Markdown.

5. **Handle Errors Gracefully**
   - If you dont have the `Answer`, inform the user that no relevant information was found, instead of making assumptions.
   - If the query is ambiguous, ask for clarification before proceeding.
"""

In [ ]:
# SYSTEM_PROMPT = """You are an intelligent assistant that can query a legal graph database (Neo4j) using `text2cypher_retriever` or `vector_cypher_retriever`. Your goal is to accurately `Answer` user queries by utilizing some tools to fetch relevant information.

# ### Instructions:
# 1. **Understand the User Query**
#    - Carefully analyze the user's question and determine the best approach to retrieve the required information.

# 2. **Use Available Tools**
#    - If the user asks **general questions** that can't be writed in Neo4j Cypher, use `vector_cypher_retriever`.
#    - If the user asks for **regulation structure, relationships, or anything that can be represented as Neo4j Cypher**, use `text2cypher_retriever`.

# 3. **How to Answer**
#    - Output tool calling to retrieve relevant data.
#    - Finally, if you know the `Answer`, then just return "KNOW" to user.

# 5. **Handle Errors Gracefully**
#    - If you dont have the `Answer`, inform the user that no relevant information was found, instead of making assumptions.
#    - If the query is ambiguous, ask for clarification before proceeding.)`
# """

In [ ]:
from IPython.display import display, Image
from langgraph.checkpoint.memory import MemorySaver

config_1 = {"configurable": {"thread_id": "1"}}
config_2 = {"configurable": {"thread_id": "2"}}
checkpointer = MemorySaver()

workflow = create_workflow(
    llm, [text2cypher_retriever, vector_cypher_retriever],
    prompt=SYSTEM_PROMPT,
    checkpointer=checkpointer,
    fallback_tool_calling=FallbackToolCalling
)

# display(Image(workflow.get_graph().draw_mermaid_png()))
# print(workflow.get_graph().draw_mermaid())
print(workflow.get_graph().draw_ascii())

In [ ]:
# Test fallback tool calling

query = "Apa isi dari pasal 100 UU no 11 tahun 2008 apa ya?"
response = workflow.invoke({"messages": query}, config_1)
display(response["messages"])
print(response["messages"][-1].content)

# for s in workflow.stream({"messages": query}, config_1, stream_mode="values"):
#     message = s["messages"][-1]
#     message.pretty_print()

In [ ]:
# Test multi tool call

query = (
    "Isi dari pasal 28 UU no 11 tahun 2008 apa ya? "
    "Kemudian carikan pasal di peraturan lain yang mana isinya mirip dengan pasal tersebut "
    "(hint pakai text2cypher RELATED_TO)"
)
response = workflow.invoke({"messages": query}, config_1)
display(response["messages"])
print(response["messages"][-1].content)

In [ ]:
prev_tool_call = AIMessage(content='', additional_kwargs={'function_call': {'name': 'text2cypher', 'arguments': '{"query": "isi dari pasal 100 UU no 11 tahun 2008"}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run-b16b6f7e-1caa-46a6-aedd-6716aeafb859-0', tool_calls=[{'name': 'text2cypher', 'args': {'query': 'isi dari pasal 100 UU no 11 tahun 2008'}, 'id': '47b70b8b-22f0-4142-91f9-6859e60100b3', 'type': 'tool_call'}], usage_metadata={'input_tokens': 393, 'output_tokens': 23, 'total_tokens': 416, 'input_token_details': {'cache_read': 0}})

tool_message = ToolMessage(content="### **Hasil Pembuatan Kode Cypher:**\n```cypher\nMATCH (r:Regulation)-[:HAS_ARTICLE]->(a:Article)\nWHERE r.type = 'UU' AND r.number = 11 AND r.year = 2008 AND a.number = '100'\nRETURN a.text AS text\n```\n\n### **Hasil Eksekusi Kode Cypher ke Database:**\nTidak dapat menemukan data yang sesuai dengan permintaan query", name='text2cypher', id='ad5822ca-c8a0-4798-a162-6f51e3dcd8f8', tool_call_id='47b70b8b-22f0-4142-91f9-6859e60100b3', artifact={'is_context_fetched': False, 'cypher_gen_usage_metadata': {'input_tokens': 2487, 'output_tokens': 66, 'total_tokens': 2553}, 'qa_usage_metadata': {'input_tokens': 0, 'output_tokens': 0, 'total_tokens': 0}, 'run_time': 0.9400525093078613})

print(FallbackToolCalling.checker(tool_message))
print(FallbackToolCalling.tool_call(prev_tool_call))

In [ ]:
query = "Apa isi pasal 28 UU No. 11 Tahun 2008? Tolong tampilkan pasal aslinya, kemudian tafsirkan isinya"
response = workflow.invoke({"messages": query}, config_2)
display(response["messages"])
print(response["messages"][-1].content)

In [ ]:
query = "Bagaimana dengan pasal setelah pasal 28 tersebut? Maksudku, carikan pasal setelah 28"
response = workflow.invoke({"messages": query}, config_2)
display(response["messages"])
print(response["messages"][-1].content)

In [ ]:
query = "Apakah kamu tahu apa kewajiban dari penyelenggara sistem elekronik?"
response = workflow.invoke({"messages": query}, config_2)
display(response["messages"])
print(response["messages"][-1].content)

In [ ]:
query = "Eh tadi isi pasal 28 dan 29 apa ya? Aku lupa (hint, tidak usah pakai tool calling)"
query = "Eh tadi isi pasal 28 dan 29 apa ya? Tadi sudah kamu jelaskan, tapi aku lupa"  # berhasil tanpa tool calling
# query = "Eh tadi isi pasal 28 dan 29 apa ya?" # Bisa salah, tapi fallback ke vector_cypher
response = workflow.invoke({"messages": query}, config_2)
display(response["messages"])
print(response["messages"][-1].content)

In [ ]:
# TUNGGU DULU BEBERAPA DETIK SEBELUM RUN KODING INI: KARENA KENA LIMIT GOOGLE GEMINI

query = "Kalau isi pasal 100 UU no 11 tahun 2008 apa ya?"
response = workflow.invoke({"messages": query}, config_2)
display(response["messages"])
print(response["messages"][-1].content)

In [ ]:
query = "Oalah begitu, memangnya berapa total pasal di UU tersebut?"
response = workflow.invoke({"messages": query}, config_2)
display(response["messages"])
print(response["messages"][-1].content)

In [ ]:
query = "Apa definisi 'data pribadi pengguna' dan pasal-pasal apa saja yang menjelaskan mengenai itu?"
response = workflow.invoke({"messages": query}, config_2)
display(response["messages"])
print(response["messages"][-1].content)